In [1]:
import numpy as np
import pandas as pd

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
from IPython.display import display

In [4]:
df=pd.read_csv('training_set_features.csv')
df2=pd.read_csv('training_set_labels.csv')

In [5]:
df2.head()

,respondent_id,xyz_vaccine,seasonal_vaccine
0,0,0,0
1,1,0,1
2,2,0,0
3,3,0,1
4,4,0,0


In [6]:
#df=pd.concat([df1.set_index('respondent_id'), df2.set_index('respondent_id')], axis=1, join='outer').reset_index()

In [7]:
#df=df.drop(columns=['employment_industry','employment_occupation'])

In [8]:
df=df.apply(lambda x: x.fillna(x.mode()[0]), axis=0)

In [9]:
X_XYZ=df.drop(columns=['employment_industry','employment_occupation','rent_or_own','marital_status'
                       ,'opinion_seas_sick_from_vacc','opinion_seas_risk','opinion_seas_vacc_effective'])

In [10]:
columns_to_encode = ['age_group','education','sex','income_poverty','employment_status','hhs_geo_region','census_msa','race']

In [11]:
df_encoded = pd.get_dummies(X_XYZ, columns=columns_to_encode).astype(float)

In [12]:
original_columns = list(X_XYZ.columns)
original_encoded_columns = [col for col in df_encoded.columns if col not in original_columns]
df_with_names = X_XYZ[original_columns].join(df_encoded[original_encoded_columns])
display(df_with_names.head())

,respondent_id,xyz_concern,xyz_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,doctor_recc_xyz,doctor_recc_seasonal,chronic_med_condition,child_under_6_months,health_worker,health_insurance,opinion_xyz_vacc_effective,opinion_xyz_risk,opinion_xyz_sick_from_vacc,age_group,education,race,sex,income_poverty,employment_status,hhs_geo_region,census_msa,household_adults,household_children,age_group_18 - 34 Years,age_group_35 - 44 Years,age_group_45 - 54 Years,age_group_55 - 64 Years,age_group_65+ Years,education_12 Years,education_< 12 Years,education_College Graduate,education_Some College,sex_Female,sex_Male,"income_poverty_<= $75,000, Above Poverty","income_poverty_> $75,000",income_poverty_Below Poverty,employment_status_Employed,employment_status_Not in Labor Force,employment_status_Unemployed,hhs_geo_region_atmpeygn,hhs_geo_region_bhuqouqj,hhs_geo_region_dqpwygqj,hhs_geo_region_fpwskwrf,hhs_geo_region_kbazzjca,hhs_geo_region_lrircsnp,hhs_geo_region_lzgpxyit,hhs_geo_region_mlyzmhmf,hhs_geo_region_oxchjgsf,hhs_geo_region_qufhixun,"census_msa_MSA, Not Principle City","census_msa_MSA, Principle City",census_msa_Non-MSA,race_Black,race_Hispanic,race_Other or Multiple,race_White
0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,1.0,2.0,55 - 64 Years,< 12 Years,White,Female,Below Poverty,Not in Labor Force,oxchjgsf,Non-MSA,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
1,1,3.0,2.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,5.0,4.0,4.0,35 - 44 Years,12 Years,White,Male,Below Poverty,Employed,bhuqouqj,"MSA, Not Principle City",0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
2,2,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,3.0,1.0,1.0,18 - 34 Years,College Graduate,White,Male,"<= $75,000, Above Poverty",Employed,qufhixun,"MSA, Not Principle City",2.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
3,3,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,3.0,3.0,5.0,65+ Years,12 Years,White,Female,Below Poverty,Not in Labor Force,lrircsnp,"MSA, Principle City",0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
4,4,2.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,3.0,2.0,45 - 54 Years,Some College,White,Female,"<= $75,000, Above Poverty",Employed,qufhixun,"MSA, Not Principle City",1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0


In [13]:
new_df=df_with_names.drop(columns=['age_group','education','sex','income_poverty','employment_status','hhs_geo_region','census_msa','race'])

In [14]:
new_df.shape

(26707, 55)

In [15]:
#x=new_df.drop(columns=['xyz_vaccine'])
#y1=new_df.loc[:,'xyz_vaccine']
y=df2.loc[:,'xyz_vaccine']

In [16]:
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import accuracy_score, classification_report

In [17]:
clf = BernoulliNB()
clf.fit(new_df,y)

BernoulliNB()

In [18]:
y1_pred = clf.predict(new_df)

In [19]:
accuracy = accuracy_score(y, y1_pred)

In [20]:
accuracy

0.8009885048863594

In [125]:
from sklearn.ensemble import RandomForestClassifier
rf_model1 = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model1.fit(new_df,y)
feature_importance = pd.DataFrame({
    'Feature': new_df.columns,
    'Importance': rf_model1.feature_importances_
})
feature_importance = feature_importance.sort_values(by='Importance', ascending=False)
print(feature_importance)

                                     Feature  Importance
0                              respondent_id    0.097754
10                           doctor_recc_xyz    0.095359
17                          opinion_xyz_risk    0.080210
16                opinion_xyz_vacc_effective    0.069071
18                opinion_xyz_sick_from_vacc    0.034119
1                                xyz_concern    0.032927
19                          household_adults    0.029056
11                      doctor_recc_seasonal    0.023638
2                              xyz_knowledge    0.023462
20                        household_children    0.023158
14                             health_worker    0.020592
7                behavioral_large_gatherings    0.016053
8                    behavioral_outside_home    0.015906
48       census_msa_MSA, Not Principle  City    0.015573
9                      behavioral_touch_face    0.015233
12                     chronic_med_condition    0.014983
4                       behavio

In [21]:
probabilities = clf.predict_proba(new_df)

In [22]:
probabilities[0:40]

array([[0.95082628, 0.04917372],
       [0.93035228, 0.06964772],
       [0.94633866, 0.05366134],
       [0.77352297, 0.22647703],
       [0.91753013, 0.08246987],
       [0.8291507 , 0.1708493 ],
       [0.9938018 , 0.0061982 ],
       [0.56853167, 0.43146833],
       [0.88601838, 0.11398162],
       [0.92826728, 0.07173272],
       [0.94960212, 0.05039788],
       [0.86448152, 0.13551848],
       [0.77271213, 0.22728787],
       [0.80810535, 0.19189465],
       [0.91345816, 0.08654184],
       [0.85718721, 0.14281279],
       [0.20439245, 0.79560755],
       [0.5796298 , 0.4203702 ],
       [0.17815215, 0.82184785],
       [0.96085266, 0.03914734],
       [0.2051535 , 0.7948465 ],
       [0.40171297, 0.59828703],
       [0.21617121, 0.78382879],
       [0.98636807, 0.01363193],
       [0.54600897, 0.45399103],
       [0.90717041, 0.09282959],
       [0.30821809, 0.69178191],
       [0.85307808, 0.14692192],
       [0.97723752, 0.02276248],
       [0.31386339, 0.68613661],
       [0.

In [23]:
y1_pred[0:40]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1,
       1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0], dtype=int64)

In [24]:
xyz_prob=[p[1] for p in probabilities]

In [25]:
xyz_prob

[0.04917371633048956,
 0.06964771898844827,
 0.05366134028419398,
 0.2264770326160334,
 0.08246986626191083,
 0.17084930022180436,
 0.006198195608364999,
 0.4314683259392755,
 0.11398161589305406,
 0.07173272036700465,
 0.050397883119377394,
 0.1355184813094532,
 0.22728786778848484,
 0.1918946450590726,
 0.08654183577730305,
 0.14281279015946374,
 0.7956075476699395,
 0.42037020498508354,
 0.821847853907273,
 0.03914734362931877,
 0.7948464975685026,
 0.5982870347895112,
 0.7838287853748959,
 0.013631934024366797,
 0.45399103295077053,
 0.09282958613883177,
 0.691781910355126,
 0.14692191594052775,
 0.02276248155869728,
 0.6861366050839984,
 0.7458733057294541,
 0.04481418577111147,
 0.018553480082906717,
 0.5155294938576797,
 0.09492941768649475,
 0.8965641213528074,
 0.3294396141696089,
 0.015988426594798166,
 0.848998136050559,
 0.1013530783796152,
 0.010041107503193552,
 0.5304455791627246,
 0.01080378531126703,
 0.6620363982951408,
 0.06816270172502638,
 0.174675313164513,
 0.037

## prediction

In [21]:
test_df=pd.read_csv('test_set_features.csv')

In [22]:
test_df=test_df.apply(lambda x: x.fillna(x.mode()[0]), axis=0)

In [23]:
test_df.isnull().sum()

respondent_id                  0
xyz_concern                    0
xyz_knowledge                  0
behavioral_antiviral_meds      0
behavioral_avoidance           0
behavioral_face_mask           0
behavioral_wash_hands          0
behavioral_large_gatherings    0
behavioral_outside_home        0
behavioral_touch_face          0
doctor_recc_xyz                0
doctor_recc_seasonal           0
chronic_med_condition          0
child_under_6_months           0
health_worker                  0
health_insurance               0
opinion_xyz_vacc_effective     0
opinion_xyz_risk               0
opinion_xyz_sick_from_vacc     0
opinion_seas_vacc_effective    0
opinion_seas_risk              0
opinion_seas_sick_from_vacc    0
age_group                      0
education                      0
race                           0
sex                            0
income_poverty                 0
marital_status                 0
rent_or_own                    0
employment_status              0
hhs_geo_re

In [24]:
XXX=test_df.drop(columns=['employment_industry','employment_occupation','rent_or_own','marital_status'
                       ,'opinion_seas_sick_from_vacc','opinion_seas_risk','opinion_seas_vacc_effective'])

In [25]:
columns_to_encode = ['age_group','education','sex','income_poverty','employment_status','hhs_geo_region','census_msa','race']

In [26]:
test_df_encoded = pd.get_dummies(XXX, columns=columns_to_encode).astype(float)

In [27]:
original_columns = list(XXX.columns)
original_encoded_columns = [col for col in df_encoded.columns if col not in original_columns]
df_with_names = XXX[original_columns].join(df_encoded[original_encoded_columns])
display(df_with_names.head())

,respondent_id,xyz_concern,xyz_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,doctor_recc_xyz,doctor_recc_seasonal,chronic_med_condition,child_under_6_months,health_worker,health_insurance,opinion_xyz_vacc_effective,opinion_xyz_risk,opinion_xyz_sick_from_vacc,age_group,education,race,sex,income_poverty,employment_status,hhs_geo_region,census_msa,household_adults,household_children,age_group_18 - 34 Years,age_group_35 - 44 Years,age_group_45 - 54 Years,age_group_55 - 64 Years,age_group_65+ Years,education_12 Years,education_< 12 Years,education_College Graduate,education_Some College,sex_Female,sex_Male,"income_poverty_<= $75,000, Above Poverty","income_poverty_> $75,000",income_poverty_Below Poverty,employment_status_Employed,employment_status_Not in Labor Force,employment_status_Unemployed,hhs_geo_region_atmpeygn,hhs_geo_region_bhuqouqj,hhs_geo_region_dqpwygqj,hhs_geo_region_fpwskwrf,hhs_geo_region_kbazzjca,hhs_geo_region_lrircsnp,hhs_geo_region_lzgpxyit,hhs_geo_region_mlyzmhmf,hhs_geo_region_oxchjgsf,hhs_geo_region_qufhixun,"census_msa_MSA, Not Principle City","census_msa_MSA, Principle City",census_msa_Non-MSA,race_Black,race_Hispanic,race_Other or Multiple,race_White
0,26707,2.0,2.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,5.0,1.0,1.0,35 - 44 Years,College Graduate,Hispanic,Female,"> $75,000",Employed,mlyzmhmf,"MSA, Not Principle City",1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
1,26708,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,1.0,1.0,18 - 34 Years,12 Years,White,Male,Below Poverty,Employed,bhuqouqj,Non-MSA,3.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
2,26709,2.0,2.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,5.0,4.0,2.0,55 - 64 Years,College Graduate,White,Male,"> $75,000",Employed,lrircsnp,Non-MSA,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
3,26710,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,4.0,2.0,2.0,65+ Years,12 Years,White,Female,"<= $75,000, Above Poverty",Not in Labor Force,lrircsnp,"MSA, Not Principle City",1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
4,26711,3.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,5.0,2.0,4.0,35 - 44 Years,12 Years,Black,Female,"<= $75,000, Above Poverty",Employed,lzgpxyit,Non-MSA,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0


In [28]:
new_test_df=df_with_names.drop(columns=['age_group','education','sex','income_poverty','employment_status','hhs_geo_region','census_msa','race'])

In [30]:
new_test_df=new_test_df.apply(lambda x: x.fillna(x.mode()[0]), axis=0)

In [31]:
predict_xyz=clf.predict_proba(new_test_df)

In [32]:
XYZ_prob=[p[1] for p in predict_xyz]

In [33]:
XYZ_prob

[0.1044127132724217,
 0.01436199349518292,
 0.13002351644223892,
 0.5080697433427144,
 0.2838465787295765,
 0.1663243456508122,
 0.636107354047915,
 0.09325364097945377,
 0.19046558400572486,
 0.08453880461253054,
 0.057397013657949005,
 0.12054620284524722,
 0.43914151339194096,
 0.07854340872688231,
 0.46883868074128143,
 0.13781454984334748,
 0.5605987379760822,
 0.03426305818017784,
 0.1232215278408709,
 0.12607055780961435,
 0.19892134417560106,
 0.1314899242231007,
 0.23027028841375705,
 0.04524951821395258,
 0.007416574845426094,
 0.2745314143749191,
 0.842685725284936,
 0.7281373493846529,
 0.03143523115523961,
 0.18681256574988536,
 0.1055750808709539,
 0.7959795657234081,
 0.007441967518790698,
 0.038520708414418986,
 0.2367672449378215,
 0.11691271424377765,
 0.08521396112171603,
 0.09686966305153881,
 0.13746524565767776,
 0.08140325906280269,
 0.04839054640145975,
 0.19794349019145832,
 0.28808009887600505,
 0.6888396454789706,
 0.33993020051865586,
 0.07316213148428156,
 

In [34]:
%store XYZ_prob

Stored 'XYZ_prob' (list)
